In [1]:
import urllib.parse
import requests
import pandas as pd
import os

In [2]:
# 19년 ~ 23년 6월
ym = [1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912,
2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
2101, 2102, 2103, 2104, 2105, 2106, 2107, 2108, 2109, 2110, 2111, 2112,
2201, 2202, 2203, 2204, 2205, 2206, 2207, 2208, 2209, 2210, 2211, 2212,
2301, 2302, 2303, 2304, 2305, 2306]

In [3]:
# 웹 크롤링으로 구한 월별 드라마배우 브랜드랭킹

brand_rank = pd.DataFrame()
error_list = []
for time in ym:
  try:
    df = pd.read_html(f'https://brikorea.com/rk/drama{time}')[0]
    df['Link'] = time
  except Exception:
    df = pd.DataFrame({'Link' : [time]})
    error_list.append(time)

  brand_rank = pd.concat([brand_rank, df])

# 아래 brand와 합치기 위해 편집
brand_rank = brand_rank.rename(columns = {'Link' : '기간'})
brand_rank = brand_rank.dropna().iloc[:, :-2]

In [4]:
pd.read_excel('/content/브랜드.xlsx').to_csv('/content/브랜드.csv', index = False, encoding = 'utf-8-sig')

In [5]:
pd.read_csv('/content/브랜드.csv')

,Unnamed: 0,Unnamed: 1,1월19일,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,기간,순위,드라마배우,참여지수,미디어지수,소통지수,커뮤니티지수,브랜드평판지수
1,1901,1,염정아,1372024,3513112,2794528,2282605,9962269
2,1901,2,현빈,2894066,2788296,1695680,2233463,9611504
3,1901,3,송혜교,1746374,2978220,3207168,1676911,9608673
4,1901,4,박보검,1933351,2855088,2242528,2345178,9376145
...,...,...,...,...,...,...,...,...
722,2009,46,이일화,602657,322812,49824,49877,1025170
723,2009,47,엄현경,232876,576828,86400,127993,1024097
724,2009,48,황승언,430782,289884,210816,66219,997700
725,2009,49,정보석,224134,599760,95616,75370,994880


In [6]:
# 노가다로 구한 월별 드라마배우 브랜드 랭킹
brand = pd.read_csv('/content/브랜드.csv')
brand.columns = brand.iloc[0,:]
brand = brand.drop(0)
brand = brand[~brand['기간'].isna()]
brand = brand.rename(columns = {'드라마배우' : '브랜드'})
brand = brand[brand['순위'] != '순위']

In [8]:
# 크롤링 + 노가다
brand_ranking = pd.concat([brand, brand_rank])

cols = brand_ranking.columns.tolist()
numcols = cols[:2] + cols[3:]

brand_ranking[numcols] = brand_ranking[numcols].apply(pd.to_numeric)


brand_ranking = brand_ranking.sort_values(by = ['기간', '순위'])
brand_ranking[numcols] = brand_ranking[numcols].astype(int)
brand_ranking

,기간,순위,브랜드,참여지수,미디어지수,소통지수,커뮤니티지수,브랜드평판지수
1,1901,1,염정아,1372024,3513112,2794528,2282605,9962269
2,1901,2,현빈,2894066,2788296,1695680,2233463,9611504
3,1901,3,송혜교,1746374,2978220,3207168,1676911,9608673
4,1901,4,박보검,1933351,2855088,2242528,2345178,9376145
5,1901,5,유이,831162,3082884,3204288,1530746,8649080
...,...,...,...,...,...,...,...,...
45,2306,46,이정진,61815,57143,79567,303056,501581
46,2306,47,송하윤,146589,130888,87304,118397,483177
47,2306,48,이원정,126062,115830,107849,80611,430352
48,2306,49,최화정,116062,103990,159521,42884,422457


In [9]:
# 데이터 다운로드
brand_ranking.to_csv('/content/배우브랜드.csv', index = False, encoding = 'utf-8-sig')

In [10]:
# 랭킹 누락 연월
off_list = sorted(list(set(ym) - set(brand_ranking['기간'].unique().tolist())))
print(len(off_list), '개월 누락 :', off_list)

25 개월 누락 : [1902, 1903, 1904, 1905, 1907, 1909, 1910, 1911, 2003, 2004, 2006, 2007, 2010, 2012, 2101, 2103, 2105, 2106, 2108, 2111, 2202, 2204, 2208, 2211, 2302]
